In [ ]:
#terminal
!

In [1]:
import speech_recognition
import tempfile
from gtts import gTTS
from pygame import mixer
from googletrans import Translator

In [2]:
# turn speech to sentences
def listenTo():
    # input from mic
    r = speech_recognition.Recognizer() 
    with speech_recognition.Microphone() as source:
        print("稍等") 
        r.energy_threshold = 4000
        r.adjust_for_ambient_noise(source, duration=1) 
        print('你想說什麼')
        speak("你想說什麼")
        audio = r.listen(source)

    # transfer input to sentences.
    try:
        print("你說:")
        print(r.recognize_google(audio, language="zh-TW"))
        
        if r.recognize_google(audio, language="zh-TW")=='離開':
            return('000-exit')
        else:
            return(r.recognize_google(audio, language="zh-TW"))
    except (speech_recognition.UnknownValueError, speech_recognition.RequestError):
        print("講清楚點再講一次,或說‘離開’結束功能[l]")
        speak("講清楚點再講一次,或說,離開,結束功能[l]")
        tmp = listenTo()
        while tmp is None:
            tmp = listenTo()
        return(tmp)

In [3]:
# turn sentences to speech
def speak(sentence, lang='zh-tw'):
    mixer.init()
    with tempfile.NamedTemporaryFile(delete=True) as fp:
        tts = gTTS(text=sentence, lang=lang)
        tts.save("{}.mp3".format(fp.name))
        mixer.music.load('{}.mp3'.format(fp.name))
        mixer.music.play()
        # continue other event after the music is done.
        while mixer.music.get_busy() == True:
            continue

In [4]:
def extract_location(sentence):
    in_index = sentence.rfind(' in ')
    if in_index==-1:
        if len(sentence.split())<3:
            in_index = sentence.lower().rfind(' weather')
            print(sentence[:in_index].strip())
            return(sentence[:in_index].strip())
        else:
            print("請只說地點")
            speak("請只說地點", 'zh-tw')
            tmp = listenTo()
            while tmp is None:
                tmp = listenTo()
        return(tmp)
    else:
        print(sentence[in_index+3:].strip())
        return(sentence[in_index+3:].strip())

In [5]:
#weather = Weather(unit=Unit.CELSIUS)
#location = weather.lookup_by_location('餔拉不拉')

In [6]:
from weather import Weather, Unit

# get weather information by weather api
def get_weather(loc):
    weather = Weather(unit=Unit.CELSIUS)
    try:
        location = weather.lookup_by_location(loc)
        
        if location is not None:
            weather_cond = translator.translate(location.location.city, dest='zh-tw').text + \
                           '今天天氣為' + translator.translate(location.forecast[0].text, dest='zh-tw').text + \
                           ', 最高溫度為' + location.forecast[0].high + '度,最低溫度' + location.forecast[0].low + '度,' + \
                           '明天天氣為' + translator.translate(location.forecast[1].text, dest='zh-tw').text + \
                           ', 最高溫度為' + location.forecast[1].high + '度,最低溫度' + location.forecast[1].low + '度'
            print(weather_cond)
            speak(weather_cond)
            #return(True)
        else:
            raise ValueError('no result return')
    except (AttributeError, ValueError):
        print("請只說地點,或說‘離開’結束功能[w]")
        speak("請只說地點,或說,離開,結束功能[w]")
        #return(None)
        #get_weather(listenTo())
        tmp = listenTo()
        if tmp=='000-exit':
            print("好吧我走[w]")
            speak("好吧我走[w]")
            return(None)
        else:
            get_weather(tmp)

In [7]:
import pandas as pd
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup

def get_yahoo_movie_info():    
    ###
    # first part
    ###
    # crawl down movie showtimes of vieshow cinemas at xinyi
    xinyi_cinemas_url = 'https://movies.yahoo.com.tw/theater_result.html/id=45'
    xinyi_page = urlopen(xinyi_cinemas_url)
    xinyi_soup = BeautifulSoup(xinyi_page, 'html.parser')
    theater_list = pd.DataFrame({
        'name':[ele.get_text() for ele in xinyi_soup.select('.theaterlist_name a')],\
        'type':[ele.get_text() for ele in xinyi_soup.select('.theaterlist_name .tapR')],\
        'time':[re.sub('\n', ',', ele.get_text().strip()) for ele in xinyi_soup.select('.theater_time')]
    })
    theater_list = theater_list[theater_list['type']=='數位']
    ###
    # second part
    ###
    # crawl down movie ranking from yahoo movies
    cinemas_rank_url = 'https://movies.yahoo.com.tw/chart.html'
    cinemas_rank_page = urlopen(cinemas_rank_url)
    cinemas_rank_soup = BeautifulSoup(cinemas_rank_page, 'html.parser')
    movie_score= pd.DataFrame({
        'name':[cinemas_rank_soup.select('h1')[-1].get_text()] + [ele.get_text() for ele in cinemas_rank_soup.select('.rank_txt')],
        'score':[ele.get_text() for ele in cinemas_rank_soup.select('.count')]
    })
    # merge movie info and showtimes
    movie_list = pd.merge(theater_list, movie_score, on='name', how='right').dropna()
    movie_list.score = pd.to_numeric(movie_list.score, errors='coerce')
    #movie_list.sort_values('score', ascending=False).drop('type', 1)
    return(movie_list)

In [8]:
movie_list = get_yahoo_movie_info()

In [9]:
import random

## recommand three movies from movie ranking data 
def random_movie():
    tmp = random.sample(list(set(movie_list.name)), 3)
    string = '給你隨機三部電影吧！' +\
        '1,' + tmp[0] + ',2,' + tmp[1] + ',3,' + tmp[2]
    print(string)
    speak(string)

In [10]:
from pypinyin import lazy_pinyin
from fuzzywuzzy import fuzz

# search movie info which is in the rank list 
# using pinyin and fuzzy match to ignore typo in chinese
movie_name_pinyin = [re.sub(' ', '', ''.join(lazy_pinyin(ele))) for ele in movie_list.name]
def movie_info(listen_movie_name, speakout=True):
    fuzzy_match_score = [fuzz.partial_ratio(''.join(lazy_pinyin(ele)), ''.join(lazy_pinyin(listen_movie_name))) for ele in movie_name_pinyin]
    
    if any(ele>60 for ele in fuzzy_match_score):
        movie_name_input = movie_list.name[fuzzy_match_score.index(max(fuzzy_match_score))]
        string = movie_name_input + ', 評分為' + re.sub('\\.', '點', str(movie_list[movie_list.name==movie_name_input]['score'].iloc[0])) + \
                 '分, 信義威秀今天上映時間為' + re.sub(':', '點', ',另一廳的時間為,'.join(movie_list[movie_list.name==movie_name_input]['time']))
        if speakout==True:
            print(string)
            speak(string)
        else:
            return(True)
    else:
        if speakout==True:
            print('查不到這電影啦')
            speak('查不到這電影啦')
        else:
            return(False)

/Users/r3dmaohong/anaconda3/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [11]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tflearn
import tensorflow as tf
import random
import json

stemmer = LancasterStemmer()

# training data
with open('intents.json') as json_data:
    intents = json.load(json_data)
words = [] # words list
classes = [] # what kind of function the ai has?
documents = [] # documents after tokenized and its label
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
# unique
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

/Users/r3dmaohong/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [12]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # one hot encoding the label
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# sort it randomly
random.shuffle(training)
training = np.array(training)

# training input and output (label)
train_x = list(training[:,0])
train_y = list(training[:,1])

In [13]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('arron.model.tflearn')

Training Step: 1999  | total loss: 0.00500 | time: 0.004s
| Adam | epoch: 1000 | loss: 0.00500 - acc: 1.0000 -- iter: 08/14
Training Step: 2000  | total loss: 0.00506 | time: 0.007s
| Adam | epoch: 1000 | loss: 0.00506 - acc: 1.0000 -- iter: 14/14
--
INFO:tensorflow:/Users/r3dmaohong/workspace/robots/aaron-personal-assistant/arron.model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [14]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [15]:
# restore all of our data structures
import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [16]:
# load our saved model
model.load('./arron.model.tflearn')

INFO:tensorflow:Restoring parameters from /Users/r3dmaohong/workspace/robots/aaron-personal-assistant/arron.model.tflearn


In [17]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return(sentence_words)

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
    return(np.array(bag))

In [18]:
# if any of the probabilities in the result is lower than error_threshold, don't extract it.
ERROR_THRESHOLD = 0.3
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list[0][0]

In [19]:
# memorized previous question type
previous_status=''

In [21]:
while True:
    string = listenTo()
    translator = Translator()
    input_status = classify(translator.translate(string, dest='en').text)
    
    if string=='000-exit':
        print("好吧我走")
        speak("好吧我走")
        break
    
    if input_status=='movie_info':
        print('喔你想查電影喔')
        speak('喔你想查電影喔')
        movie_info(string)
    elif ((previous_status=='movie_info')) & (movie_info(string, speakout=False)):
        print('喔你想查電影喔')
        speak('喔你想查電影喔')
        movie_info(string)
    elif input_status=='random_movie':
        print('好吧推薦電影給你')
        speak('好吧推薦電影給你')
        random_movie()
    elif input_status=='weather':
        if string=='000-exit':
            pass
        else:
            en_input_string = translator.translate(string, dest='en').text
            loc = extract_location(en_input_string)
            get_weather(loc)
    else:
        print('不知道你在說啥')
        speak('不知道你在說啥')
    previous_status = input_status

稍等
你想說什麼
你說:
天氣如何
請只說地點
稍等
你想說什麼
你說:
台北
台北市今天天氣為雷暴, 最高溫度為30度,最低溫度22度,明天天氣為雷暴, 最高溫度為31度,最低溫度23度
稍等
你想說什麼
你說:
責任推薦我的電影好了
好吧推薦電影給你
給你隨機三部電影吧！1,糯爾摩斯,2,一級玩家,3,雨妳再次相遇
稍等
你想說什麼
你說:
我要查電影一級玩家的相關資訊
喔你想查電影喔
一級玩家, 評分為4點7分, 信義威秀今天上映時間為09點30,10點50,11點30,12點10,13點30,14點50,16點10,16點50,17點30,18點50,20點10,21點30,22點50,00點10,00點50,01點30
稍等
你想說什麼
你說:
講清楚點再講一次,或說‘離開’結束功能[l]
稍等
你想說什麼
你說:
我想知道洪雪嗎
喔你想查電影喔
紅雀, 評分為4點4分, 信義威秀今天上映時間為21點05
稍等
你想說什麼
你說:
離開
好吧我走


In [ ]:
string

In [ ]:
nltk.download('punkt')

In [ ]:
fuzzy_match_score.index(max(fuzzy_match_score))

In [ ]:
[''.join(lazy_pinyin(ele)) for ele in movie_list.name]

In [ ]:
speak('11:50,14:00,16:10,20:20,22:30,00:40', 'zh-tw')

In [ ]:
xinyi_soup.findAll('h1')

In [ ]:
len(xinyi_soup.select('.theaterlist_name a'))

In [ ]:
listenTo()

In [ ]:
speak(get_weather('taipei'), 'zh-tw')

In [ ]:
translator.translate('台北', dest='zh-tw').text

In [ ]:
extract_location(en_input_string)

In [ ]:
from weather import Weather, Unit
weather = Weather(unit=Unit.CELSIUS)

location = weather.lookup_by_location('djsjdisjd')
location.condition.text

In [ ]:
location.forecast[0].date

In [ ]:
print('最高溫度', location.forecast[0].high, '最低溫度', location.forecast[0].low)

In [ ]:
location.print_obj